In [1]:
%matplotlib inline
from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
from astropy.utils.data import get_pkg_data_filename

In [2]:
#start of bruno's code

import subprocess as sp

import glob

import astropy.io.fits as pyfits


#a fcn to create a file to use for PARAMETERS_NAME input to use for SExtractor
def scampparams():
    #list of parameters for SExtractor we want
    pars = ["XWIN_IMAGE", "YWIN_IMAGE", "ERRAWIN_IMAGE", "ERRBWIN_IMAGE",\

            "ERRTHETAWIN_IMAGE", "FLUX_AUTO", "FLUXERR_AUTO", "FLAGS",\

            "FLAGS_WEIGHT", "FLUX_RADIUS", "MAG_AUTO","ALPHA_J2000","DELTA_J2000"]



    f = open("scamp.param","w")

    for p in pars:

        f.write(f"{p}\n")

    f.close()

    return None


In [3]:
#excutes the fcn above to ACTUALLY generate the file
scampparams()

# SANITY check of Zeropoint calculations
#### note that the below test example is just a weird one?  header keywords & values arent there for some reason... ...maybe b/c this was a "total" file?

In [4]:
#looking at hst fits data to get values needed for remaining zeropt calculations (my code)--(test1_image)

#verifying calculation matches known value from bruno
test1_file = fits.open('/data1/rowland/fits_data/hst_14096_i1_acs_wfc_total_drz.fits') #uses acs f435w
#test_file.info #shows theres 1 header, 4 images, 1 BinTable images - well [1] should be the SCI image then


test1_header0 = test1_file[0].header #header
''' 
test1_header1 = test1_file[1].header #science image 'SCI'
test1_header2 = test1_file[2].header #weights image 'WHT'
test1_header3 = test1_file[3].header #something we wont use image 'CTX'
test1_header4 = test1_file[4].header #wait where did this come from 'EXP'
test1_header5 = test1_file[5].header #whatever a bintable is 'HDRTAB' think it formats header list
''' 
#print(test1_header1['PHOTFLAM'])

#test1_file[0].header #goes from 0-5
#test1_header0["FILTER"] #NOT FOUND
test1_header0["FILTER2"] #returns 'F435W'


'F435W'

In [5]:
#trying another file (test2 image) b/c maybe its just that one? ------that one IS just weird....--test2 image
#verifying calculation matches known value from bruno
test2_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f814w_drz.fits') #uses acs f435w
#test_file.info #shows theres 1 header, 4 images, 1 BinTable images - well [1] should be the SCI image then

test2_file[0].header #goes from 0-4

test2_header0 = test2_file[0].header #header
test2_header1 = test2_file[1].header #science image 'SCI'
test2_header2 = test2_file[2].header #weights image 'WHT'
test2_header3 = test2_file[3].header #something we wont use image 'CTX'
test2_header4 = test2_file[4].header #wait where did this come from 'EXP'
 
#print('PHOTFLAM:',test2_header1['PHOTFLAM'])
#print('PHOTPLAM:',test2_header1['PHOTPLAM'])

PHOTFLAM_test2 = test2_header1['PHOTFLAM'] #7.0073061E-20 / inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_test2 = test2_header1['PHOTPLAM'] #8.0593491E+03 / Pivot wavelength (Angstroms)
#test2_header0["FILTER"] #NOT FOUND
#print('FILTER2:',test2_header0["FILTER2"]) #returns 'F814W'
#print('FILTER1:',test2_header0['FILTER1']) #returns 'CLEAR1L'

In [6]:
#ZEROpoint calculation for test2

ZP_f814w = (-2.5 * np.log10(PHOTFLAM_test2)) - (5 * np.log10(PHOTPLAM_test2)) - 2.408
print(ZP_f814w, 'should match expected value of:', "f814w:25.947")
print('this is good!')

25.94662243630965 should match expected value of: f814w:25.947
this is good!


# NOTE TO SELF: 
### [for trying to automate selections, idk why but filter changes from 1 to 2 for correct name (see above test2 & below f850lp for an example)]

# Calculating ZEROpoints of remaining filters

In [7]:
#calculating remaining 3 zeropoints (f850lp, f775w, f625w)


#first f850lp
f850lp_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f850lp_drz.fits')
f850lp_header0 = f850lp_file[0].header #header

f850lp_header1 = f850lp_file[1].header #science image 'SCI'
f850lp_header2 = f850lp_file[2].header #weights image 'WHT'
f850lp_header3 = f850lp_file[3].header #something we wont use image 'CTX'
f850lp_header4 = f850lp_file[4].header #wait where did this come from 'EXP'
#f850lp_header5 = f850lp_file[5].header #whatever a bintable is 'HDRTAB' think it formats header list

#print(f850lp_header1['PHOTFLAM']) 
#f850lp_file.info
#f850lp_file[0].header #goes from 0-4
#f850lp_header0["FILTER"] #NOT FOUND
#print(f850lp_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f850lp_header0["FILTER1"])#returns 'F850LP'

#needed values
PHOTFLAM_f850lp = f850lp_header1['PHOTFLAM']#1.5213061E-19 #/ inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_f850lp = f850lp_header1['PHOTPLAM']#9.0336270E+03 #/ Pivot wavelength (Angstroms)   

FILTER1: F850LP


In [8]:
#second f775w

f775w_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f775w_drz.fits')

#f775w_file.info
#f775w_file[0].header #goes from 0-4

f775w_header0 = f775w_file[0].header #header
f775w_header1 = f775w_file[1].header #science image 'SCI'
f775w_header2 = f775w_file[2].header #weights image 'WHT'
f775w_header3 = f775w_file[3].header #something we wont use image 'CTX'
f775w_header4 = f775w_file[4].header #wait where did this come from 'EXP'

##print('PHOTFLAM:',f775w_header1['PHOTFLAM']) 
#f775w_header1

#print('FILTER:',f775w_header0["FILTER"]) #NOT FOUND
print('FILTER2:',f775w_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f775w_header0["FILTER1"])#returns 'F775W'

#needed values
PHOTFLAM_f775w = f775w_header1['PHOTFLAM']#9.9942165E-20 / inverse sensitivity, ergs/cm2/Ang/electron
PHOTPLAM_f775w = f775w_header1['PHOTPLAM']#7.6936714E+03 / Pivot wavelength (Angstroms) 

FILTER2: CLEAR2L
FILTER1: F775W


In [9]:
#third f625w

f625w_file = fits.open('/data1/rowland/fits_data/hst_mos_1041271_acs_wfc_f625w_drz.fits')

#f625w_file.info
###f625w_file[4].header #goes from 0-4

f625w_header0 = f625w_file[0].header #header
f625w_header1 = f625w_file[1].header #science image 'SCI'
f625w_header2 = f625w_file[2].header #weights image 'WHT'
f625w_header3 = f625w_file[3].header #something we wont use image 'CTX'
f625w_header4 = f625w_file[4].header #wait where did this come from 'EXP'

print('PHOTFLAM:',f625w_header1['PHOTFLAM']) 
print('PHOTPLAM:',f625w_header1['PHOTPLAM'])
#f625w_header1

#print('FILTER:',f625w_header0["FILTER"]) #NOT FOUND
print('FILTER2:',f625w_header0["FILTER2"]) #returns 'CLEAR2L'
print('FILTER1:',f625w_header0["FILTER1"])#returns 'F625W'

#needed values
PHOTFLAM_f625w = f625w_header1['PHOTFLAM']#1.1931234E-19 / inverse sensitivity, ergs/cm2/Ang/electron 
PHOTPLAM_f625w = f625w_header1['PHOTPLAM']#6.3120513E+03 / Pivot wavelength (Angstroms) 

PHOTFLAM: 1.1931234e-19
PHOTPLAM: 6312.0513
FILTER2: CLEAR2L
FILTER1: F625W


In [10]:
photflam_list = [PHOTFLAM_f850lp, PHOTFLAM_f775w, PHOTFLAM_f625w]
photplam_list = [PHOTPLAM_f850lp, PHOTPLAM_f775w, PHOTPLAM_f625w]
#phot_list = [(PHOTFLAM_test2, PHOTPLAM_test2)] #tested for proof of concept - worked!
phot_list = [(PHOTFLAM_f850lp, PHOTPLAM_f850lp), (PHOTFLAM_f775w, PHOTPLAM_f775w), (PHOTFLAM_f625w, PHOTPLAM_f625w)]
ZPs = []
for x,y in phot_list:
    Zeropts = (-2.5 * np.log10(x)) - (5 * np.log10(y)) - 2.408
    ZPs.append(Zeropts)
ZPs = np.array(ZPs)
print('in the order of f850lp, f775w, f625w, the zeropts are:',ZPs)

in the order of f850lp, f775w, f625w, the zeropts are: [24.85714771 25.66195995 25.89943399]


# Making dictionaries with all filter values included

In [11]:
#creating needed dictionaries with calculated zeropoints included

#need a dictionary for zeropoint values of each filter
zeropoints = {"F435W":25.665,"F606W":26.493,"F814W":25.97,"F105W":26.2687,"F125W":26.2303,"F140W":24.4524,\
              "F160W":25.9463, "F625W":24.85714771, "F775W":25.66195995, "F850LP":25.89943399}

#need to create a dictionary for the 10 indiviual filters
filters_used = {'F435W','F606W','F814W','F625W','F775W','F850LP','F105W','F125W','F140W','F160W'}

# SANITY  check of splitting 'sci' & 'wht' frames

In [12]:
#testing what values for loop gathers --test3 image

test3_image = fits.open('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits') # equal to 'hdu' in for loop

test3_header = test3_image[0].header 
test3_filename = test3_header["FILTER"] #so this will work for SE files, prob not NW ones tho
print('test3_filename:',test3_filename) #seeing if this worked
test3_gain = float(test3_image[0].header["CCDGAIN"])
print('test3_gain:',test3_gain) #not sure what this means....
test3_exptime = float(test3_image[0].header['EXPTIME'])
print('test3_exptime:',test3_exptime) #exposure time
#other stuff in code
test3_headerOut = test3_image["SCI"].header
#this part builds a header from available info if filter name doesnt match list
    #print(test_headerOut) #prints the science frame header for each file
    #lines after that pull values from frame 0 to add to science frame header
    #outname uses science frame location as name of 
test3_dataOUt = test3_image["SCI"].data/test3_gain
#print(test_dataOUt) #wtf is this?
#for i in test_dataOUt: 
#    print(i)
print('shape of test3_dataOUt:',np.shape(test3_dataOUt)) #(2069, 2124)
print('shape of test3_image.data:',np.shape(test3_image["SCI"].data)) #(2069,2124) (#arrays,len of each array) #all the data is 0 matrices tho
test3_image['SCI'].data[0]

test3_filename: F160W
test3_gain: 2.5
test3_exptime: 1005.872924
shape of test3_dataOUt: (2069, 2124)
shape of test3_image.data: (2069, 2124)


array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [13]:
#figured out error - below code splits & rewrites files into original & sci & sci.wht files--test4 image 

#why getting no "CCDGAIN" errors now
#somehow (overwrite below?) some filenames changed to "_sci.wht.fits" to end of filenames
test4_SEfile = fits.open('/data1/rowland/fits_data/fits_data_Doubles_SE/hst_14096_j2_wfc3_ir_f125w_drz_SE.fits')
#test_SEfile[0].header
test4_SEfile.info

<bound method HDUList.info of [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f477541a0f0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f47754318d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f4775436e80>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f4775443a20>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f4775400710>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7f4775408a90>]>

In [14]:
#example header to see if keywords 'FILTER', 'FILTER1', 'FILTER2' exist
example = fits.open('/data1/rowland/fits_data/fits_data_Doubles_SE/hst_14096_j2_wfc3_ir_f105w_drz_SE.fits')
example[0].header


SIMPLE  =                    T / Fits standard                                  
BITPIX  =                   16 / Bits per pixel                                 
NAXIS   =                    0 / Number of axes                                 
EXTEND  =                    T / File may contain extensions                    
ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        
DATE    = '2017-09-01         ' / Date FITS file was generated                  
IRAF-TLM= '2017-09-01T21:09:17' / Time of last modification                     
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 
NEXTEND =                    4 / Number of standard extensions                  
FILENAME= 'hst_14096_j2_wfc3_ir_f105w_run_1_drz.fits' / name of file            
FILETYPE= 'SCI      '          / type of data found in data file                
                            

# Splitting 'sci' & 'wht' files

In [15]:
# need to split 3 files from hst fits data files for sextractor intake reasons
# CAN ONLY RUN THIS ONCE B/C IT REWRITES FILES AFTER SEPARATING! will get error if done more than once
filterImages = {}
#only doing on doubles/SE files first to test on smaller dataset and its in a different location anyway
all_SE_files = glob.glob('/data1/rowland/all_fits_data/hst*.fits')

for image in all_SE_files:
    #if image == "%s_SE"%(image[:-5]): #"%sdrz"%(image[:-5]): #use this part for none SE files
        print('this is image',image)

        hdu = pyfits.open(image)

        header = hdu[0].header
        
        f = ""
        f1 = ""
        f2 = ""
        try:
            f = header["FILTER"] 
        except KeyError:
                pass
        try: 
             f1 = header["FILTER1"]
        except KeyError: 
                    pass
        try:
            f2 = header["FILTER2"]
        except KeyError:
                pass
            
        if f in filters_used:
            filterName = f
        elif f1 in filters_used:
            filterName = f1
        elif f2 in filters_used:
            filterName = f2 
        else:
            print('error determining filter used')
            continue
            
        gain = 2#float(hdu[0].header["CCDGAIN"])

        exptime = float(hdu[0].header["EXPTIME"])



        if filterName not in filterImages.keys():

            filterImages[filterName] = []



        print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations

        headerOut = hdu["SCI"].header

        headerOut["FILTER"] = filterName
        headerOut["RADESYS"] = 'ICRS'
        headerOut["EXPTIME"] = exptime

        headerOut["GAIN"] = gain



        name = "/data1/rowland/all_fits_data/sci_n_wht_files/"

        outname = name + image.split("/")[-1][:-5]+"_sci.fits" 

        dataOUt = hdu["SCI"].data/gain
        

        pyfits.writeto(outname,dataOUt,header=headerOut,overwrite=True) #b/c overwrite cant repeat this cell...

        pyfits.writeto(name + image.split("/")[-1][:-5]+"_sci.wht.fits",hdu["WHT"].data,header=hdu["WHT"].header,overwrite=True)



        filterImages[filterName].append(outname)

        # break

print('FILTER IMAGES printout:',filterImages)

this is image /data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits
/data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f105w_drz_SE.fits
/data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f140w_drz.fits
/data1/rowland/all_fits_data/hst_14096_i2_wfc3_ir_f140w_drz_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f140w_drz_SE.fits
/data1/rowland/all_fits_data/hst_14096_j2_wfc3_ir_f140w_drz_SE_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_i4_wfc3_ir_f105w_drz.fits
/data1/rowland/all_fits_data/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_j4_wfc3_ir_f160w_drz_SE.fits
/data1/rowland/all_fits_data/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
this is image /data1/rowland/all_fits_data/hst_14096_i4_wfc3_ir_f160w_drz.fits
/data1/rowland/

In [16]:
print('FILTER IMAGES printout:',filterImages['F105W'])

FILTER IMAGES printout: ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f105w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f105w_drz_sci.fits']


# SANITY check SExtractor start

In [17]:
### test 3 START
#error was test3_filterImages needs to begin as an empty dict before this for loop populates it

#testing below SExtractor run by first getting single file data in similar form---test3 image reused

single_test3_file = glob.glob('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits') #single file to test on
test3_filterImages = {} #{'f160w':'F160W'} #short dictionay of filters for a single file to test on


for image in single_test3_file:
    #gathers header info we want

    hdu3 = pyfits.open(image)

    header3 = hdu3[0].header
    #if 
    filterName3 = header3["FILTER"] #not sure this will work given above "NOTE TO SELF": fix = conditional if stmts

    gain3 = float(hdu3[0].header["CCDGAIN"])

    exptime3 = float(hdu3[0].header["EXPTIME"])


    #saves header info we want to each file (sci, wht) after separating out from original compressed file 
    if filterName3 not in test3_filterImages.keys():

        test3_filterImages[filterName3] = []



    print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations

    headerOut3 = hdu3["SCI"].header

    headerOut3["FILTER"] = filterName3

    headerOut3["EXPTIME"] = exptime3

    headerOut3["GAIN"] = gain3



    outname3 = "%s_sci.TEST3.fits"%(image[:-5])

    dataOUt3 = hdu3["SCI"].data/gain3

    pyfits.writeto(outname3,dataOUt3,header=headerOut3,overwrite=True) #b/c overwrite cant repeat this cell...

    pyfits.writeto("%s_sci.wht.TEST3.fits"%(image[:-5]),hdu3["WHT"].data,header=hdu3["WHT"].header,overwrite=True)



    test3_filterImages[filterName3].append(outname3)

    # break

print('test3_FILTER_IMAGES printout:',test3_filterImages) #good prints out single file so i can test below w/o crashing cpu

/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
test3_FILTER_IMAGES printout: {'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}


In [18]:
#test3_filterImages[filterName3] #prints ['filename.fits']
test3_filterImages

{'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}

In [19]:
#running test3 image outside of the for loop below so can compare anwsers

test3_thresh = 20.0
test3_sci = fits.open('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz.fits')#_sci.TEST3.fits')

#simple entries based on gal.fit example with SEx

#these gives errors
#! sex -PARAMETERS_NAME default.param hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits -DETECT_THRESH {test3_thresh} #error opening file
#! sex -PARAMETERS_NAME default.param {test3_sci} -DETECT_THRESH {test3_thresh} #errors finding file

#this way works
! sex -PARAMETERS_NAME default.param /data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits -DETECT_THRESH {test3_thresh} #this one works!

> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:04:00 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at lin

In [20]:
#trying above as a cycle now so for loop will work

test3_sci_cycle = ('/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits')
                  #/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3
! sex -PARAMETERS_NAME default.param {test3_sci_cycle} -DETECT_THRESH {test3_thresh} #works!

> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:04:00 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at lin

> Line: 2050  Objects:      297 detected /      176 sextracted
      Objects: detected 297      / sextracted 293             

> Closing files
> 
> All done (in 0.2 s: 9385.3 lines/s , 1329.1 detections/s)


In [21]:
#trying above again but trying to save outputs

test3_SEoutput = ! sex -PARAMETERS_NAME default.param {test3_sci_cycle} -DETECT_THRESH {test3_thresh} 
#print('this is my printout:', test3_SEoutput) #weird format of print out but it did work

In [22]:
#testing below on single file first --test3 image reused
test3_thresh = 20.0

test3_skycat ="SDSS-R9"

for filter in test3_filterImages.keys(): #keys come after: (name:keys)

    test3_catalogs = ""

    for image in test3_filterImages[filter]:
        #print('image:', image)#?? gave 5 iterations to print F160W separately...weird

        test3_root = (image[:-5]) #this is the "..._sci.TEST3" file that was written during above for loop, not sure why the [:-5] tho
        print('this is test3_root:',test3_root) #ERROR B/C DOESNT END WITH '.FITS'
        
        ####adding .fits to end so !sex will work......alternatively just use image!!! 
        ####test3_fn = "%s.fits"%(image[:-5])
        ####print('hope this works:',test3_fn)
        
        #not sure how this gets inputted, maybe add to ! sex -PARAMETERS_NAME....line?
        test3_catalogs += f" {test3_root}.cat" #possibly CATALOG_NAME parameter using but why 5 printouts?
        #print(test3_catalogs) #??? no clue, just prints 5 rows of '.cat', with increasing array lengths 
        
        ####! sex -PARAMETERS_NAME default.param {test3_fn} -DETECT_THRESH {test3_thresh} #this works now!
        ! sex -PARAMETERS_NAME default.param {image} -DETECT_THRESH {test3_thresh} #this works now!


this is test3_root: /data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3
> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:04:04 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits

> WARNING: Using ICRS instead of unknown astrometric reference frame: OTHER

----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at l

      Objects: detected 297      / sextracted 293             

> Closing files
> 
> All done (in 0.2 s: 9799.3 lines/s , 1387.7 detections/s)


In [23]:
#exploring test3_filterImages to get a sense of why for loop does 5 iterations in below cell 


#print(test3_filterImages) #{'f160w': 'F160W', 'F160W': ['/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits']}

#for i in test3_filterImages:
#    print(i) #(prints)f160w (next row) F160W

#type(test3_filterImages) #dict

#np.shape(test3_filterImages) #()

#len(test3_filterImages) #2

#test3_filterImages.header #dict so doesnt have, cant be a fits anymore

#test3_filterImages??

for image in test3_filterImages[filter]:
    print(image) #only single file pathway prints
    #print(image[:-5]) #file pathway w/o .fits (aka the last5 indices) obviously

/data1/rowland/fits_data/hst_14096_i2_wfc3_ir_f160w_drz_sci.TEST3.fits


In [24]:
print(filterImages.keys())
#print(filterImages.values())

dict_keys(['F160W', 'F105W', 'F140W', 'F606W', 'F125W', 'F814W', 'F775W', 'F625W', 'F435W', 'F850LP'])


# SExtractor start

In [41]:
# 2nd part of Bruno's code that actually runs sextractor AMENDED by me to run in jupyter nb

thresh = 50.0

skycat = "SDSS-R9"#"2MASS"#"USNO-B1"#
#filterImages comes from way above for loop of grouped files per filter

for filter in filterImages.keys(): #keys come after: (name:keys)

    catalogs = ""

    for image in filterImages[filter]:
        print('danielles images:',image)
        root = image[:-5]

        catalogs += f" {root}.cat"

        ! sex  {image} -DETECT_THRESH {thresh} -CATALOG_TYPE FITS_LDAC -CATALOG_NAME {root}.cat -CHECKIMAGE_TYPE NONE -PARAMETERS_NAME scamp.param -WEIGHT_TYPE NONE -FILTER Y -FILTER_NAME gauss_1.5_3x3.conv -SEEING_FWHM 0.5 -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh} -DETECT_THRESH {thresh}
       # print(test_commandSEx)
        #! sex -PARAMETERS_NAME default.param {image} -DETECT_THRESH {thresh} #this works now!
        #break 
    #break
        #sp.call(commandSEx,shell=True)
   # ! scamp {catalogs} -ASTREF_CATALOG {skycat}

    #print(commandSCAMP)
    all_images = " ".join(filterImages[filter])
    print('danielles test:',len(all_images))
    #sp.call(commandSCAMP,shell=True)
    ! swarp -IMAGEOUT_NAME /data1/rowland/elgordo_{filter}.fits -COMBINE_TYPE SUM -PIXELSCALE_TYPE MANUAL -PIXEL_SCALE 0.06 -GAIN_KEYWORD GAIN -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_SUFFIX .wht.fits {all_images}

    #break


    ##print(command)

    ##sp.call(command,shell=True)

danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:28:50 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
----- Measuring from: hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768


danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:28:50 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
----- Measuring from: hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at li

danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:28:51 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_i4_wfc3_ir_f160w_drz_sci.fits
----- Measuring from: hst_14096_i4_wfc3_ir_f160w_drz_sci.fits
      "Unnamed" / no ext. header / 2124x2069 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768


      Objects: detected 175      / sextracted 173             

> Closing files
> 
> All done (in 0.3 s: 6691.7 lines/s , 559.5 detections/s)
danielles images: /data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits
> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:28:52 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing catalog
> Looking for hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits
----- Measuring from: hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits
      "Unnamed" / no ext. header / 2123x2068 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up back

danielles test: 345

> WARNING: default.swarp not found, using internal defaults

> 
----- SWarp 2.38.0 started on 2019-06-17 at 15:28:52 with 4 threads

> Examining input data ...
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.fits ...
> Looking for hst_14096_i2_wfc3_ir_f160w_drz_sci.wht.fits ...
> Looking for hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits ...
> Looking for hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.wht.fits ...
> Looking for hst_14096_i4_wfc3_ir_f160w_drz_sci.fits ...
> Looking for hst_14096_i4_wfc3_ir_f160w_drz_sci.wht.fits ...
> Looking for hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits ...
> Looking for hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.wht.fits ...
> Creating NEW output image ...
> Creating NEW weight-map ...
> 
------- Output File elgordo_F160W.fits:
    "no ident"  WEIGHTED  no ext. header  4538x4824  32 bits (floats)
    Center: 01:02:56.30 -49:15:39.9   4.54'x4.82'  Scale: 0.06 ''/pixel
    Gain: 0 e-/ADU   Flux scaling (astrom/photom): 1 X / 1 X

> Loading input da

...

> Resampling line:   3648 / 8070   
> Resampling line:   3654 / 8070   
> Resampling line:   3660 / 8070   
> Resampling line:   3666 / 8070   
> Resampling line:   3672 / 8070   
> Resampling line:   3678 / 8070   
> Resampling line:   3684 / 8070   
> Resampling line:   3690 / 8070   
> Resampling line:   3696 / 8070   
> Resampling line:   3702 / 8070   
> Resampling line:   3708 / 8070   
> Resampling line:   3714 / 8070   
> Resampling line:   3720 / 8070   
> Resampling line:   3726 / 8070   
> Resampling line:   3732 / 8070   
> Resampling line:   3738 / 8070   
> Resampling line:   3744 / 8070   
> Resampling line:   3750 / 8070   
> Resampling line:   3756 / 8070   
> Resampling line:   3762 / 8070   
> Resampling line:   3768 / 8070   
> Resampling line:   3774 / 8070   
> Resampling line:   3780 / 8070   
> Resampling line:   3786 / 8070   
> Resampling line:   3792 / 8070   
> Resampling line:   3798 / 8070   
> Resampling line:   3804 / 8070   
> Resampling line:   3810 / 

> Resampling line:   4770 / 8070   
> Resampling line:   4776 / 8070   
> Resampling line:   4782 / 8070   
> Resampling line:   4788 / 8070   
> Resampling line:   4794 / 8070   
> Resampling line:   4800 / 8070   
> Resampling line:   4806 / 8070   
> Resampling line:   4812 / 8070   
> Resampling line:   4818 / 8070   
> Resampling line:   4824 / 8070   
> Resampling line:   4830 / 8070   
> Resampling line:   4836 / 8070   
> Resampling line:   4842 / 8070   
> Resampling line:   4848 / 8070   
> Resampling line:   4854 / 8070   
> Resampling line:   4860 / 8070   
> Resampling line:   4866 / 8070   
> Resampling line:   4872 / 8070   
> Resampling line:   4878 / 8070   
> Resampling line:   4884 / 8070   
> Resampling line:   4890 / 8070   
> Resampling line:   4896 / 8070   
> Resampling line:   4902 / 8070   
> Resampling line:   4908 / 8070   
> Resampling line:   4914 / 8070   
> Resampling line:   4920 / 8070   
> Resampling line:   4926 / 8070   
> Resampling line:   4932 / 

> Resampling line:   5904 / 8070   
> Resampling line:   5910 / 8070   
> Resampling line:   5916 / 8070   
> Resampling line:   5922 / 8070   
> Resampling line:   5928 / 8070   
> Resampling line:   5934 / 8070   
> Resampling line:   5940 / 8070   
> Resampling line:   5946 / 8070   
> Resampling line:   5952 / 8070   
> Resampling line:   5958 / 8070   
> Resampling line:   5964 / 8070   
> Resampling line:   5970 / 8070   
> Resampling line:   5976 / 8070   
> Resampling line:   5982 / 8070   
> Resampling line:   5988 / 8070   
> Resampling line:   5994 / 8070   
> Resampling line:   6000 / 8070   
> Resampling line:   6006 / 8070   
> Resampling line:   6012 / 8070   
> Resampling line:   6018 / 8070   
> Resampling line:   6024 / 8070   
> Resampling line:   6030 / 8070   
> Resampling line:   6036 / 8070   
> Resampling line:   6042 / 8070   
> Resampling line:   6048 / 8070   
> Resampling line:   6054 / 8070   
> Resampling line:   6060 / 8070   
> Resampling line:   6066 / 

> Resampling line:   7026 / 8070   
> Resampling line:   7032 / 8070   
> Resampling line:   7038 / 8070   
> Resampling line:   7044 / 8070   
> Resampling line:   7050 / 8070   
> Resampling line:   7056 / 8070   
> Resampling line:   7062 / 8070   
> Resampling line:   7068 / 8070   
> Resampling line:   7074 / 8070   
> Resampling line:   7080 / 8070   
> Resampling line:   7086 / 8070   
> Resampling line:   7092 / 8070   
> Resampling line:   7098 / 8070   
> Resampling line:   7104 / 8070   
> Resampling line:   7110 / 8070   
> Resampling line:   7116 / 8070   
> Resampling line:   7122 / 8070   
> Resampling line:   7128 / 8070   
> Resampling line:   7134 / 8070   
> Resampling line:   7140 / 8070   
> Resampling line:   7146 / 8070   
> Resampling line:   7152 / 8070   
> Resampling line:   7158 / 8070   
> Resampling line:   7164 / 8070   
> Resampling line:   7170 / 8070   
> Resampling line:   7176 / 8070   
> Resampling line:   7182 / 8070   
> Resampling line:   7188 / 

> Co-adding line:   3640 / 8473   
> Writing   line:   3640 / 8473   
> Preparing line:   5360 / 8473   
> Reading   line:   5360 / 8473   
> Co-adding line:   5360 / 8473   
> Writing   line:   5360 / 8473   
> Preparing line:   7080 / 8473   
> Reading   line:   7080 / 8473   
> Co-adding line:   7080 / 8473   
> Writing   line:   7080 / 8473   
> Closing files ...
> 
> All done (in 44.7 s)


In [26]:
filterImages['F814W']

['/data1/rowland/all_fits_data/sci_n_wht_files/hst_12477_35_acs_wfc_total_drz_sci.fits',
 '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits']

# Miscellaneous testing

In [27]:
#print({root})
#print(np.shape(all_images)) #len 345
print(all_images)

/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits


In [28]:
print(filterImages)

{'F160W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits'], 'F105W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f105w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f105w_drz_sci.fits'], 'F140W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f140w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f140w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f140w

In [29]:
print(filterImages[filter])

['/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits']


In [30]:
for filter in filterImages:
    print(filter)

F160W
F105W
F140W
F606W
F125W
F814W
F775W
F625W
F435W
F850LP


In [31]:
for image in filterImages:
    print(image)

F160W
F105W
F140W
F606W
F125W
F814W
F775W
F625W
F435W
F850LP


In [32]:
for image in filterImages['F160W']:
    print(image)

/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits


In [33]:
acs_files = {'F606W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits', \
              'F625W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits', \
              'F775W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits', \
              'F814W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits', \
              'F850LP': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits'}



In [34]:
#dictionary key:value

for filter,image in acs_files.items(): #acs_files.values(): #finally this gives datapaths #acs_files.keys():#this prints filters#acs_files.items():#prints whole dict
    print(image)
    print(filter) #acs_files.get())
    print('starting next cycle')

/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits
F606W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits
F625W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits
F775W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits
F814W
starting next cycle
/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits
F850LP
starting next cycle


# Running SExtractor on ACS filters to generate stars for PSF

In [35]:
''' 
acs_files = {'F606W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits', \
              'F625W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits', \
              'F775W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits', \
              'F814W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits', \
              'F850LP': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits'}


thresh_test = 20.0

for filter,image in acs_files.items():
    print('processing this image',image)
    #NOTE output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files
    ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}
    #break
    '''

" \nacs_files = {'F606W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f606w_drz_sci.fits',               'F625W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f625w_drz_sci.fits',               'F775W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f775w_drz_sci.fits',               'F814W': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f814w_drz_sci.fits',               'F850LP': '/data1/rowland/all_fits_data/sci_n_wht_files/hst_mos_1041271_acs_wfc_f850lp_drz_sci.fits'}\n\n\nthresh_test = 20.0\n\nfor filter,image in acs_files.items():\n    print('processing this image',image)\n    #NOTE output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files\n    ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]

# Running SExtractor on WFC filters to generate stars for PSF

In [36]:
'''
wfc_files = {'F105W':['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f105w_drz_SE_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f105w_drz_sci.fits'], \
            'F125W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f125w_drz_SE_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f125w_drz_SE_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f125w_drz_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f125w_drz_sci.fits'], \
            'F140W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f140w_drz_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f140w_drz_SE_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f140w_drz_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f140w_drz_SE_sci.fits'], \
            'F160W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits', \
             '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits']}
thresh_test = 20.0


# NOTE BEFORE RUNNING BELOW TO FIX BRUNOS CODE TO HANDLE INPUTS, MAYBE WANNA CHANGE NAME OF testPSF_filter? so it wont rewrite






#for filter,image in wfc_files.items():
#    print('processing this image',image)
    #NOTE output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files
#    ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}
#    break
'''

"\nwfc_files = {'F105W':['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits',              '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f105w_drz_sci.fits',              '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f105w_drz_SE_sci.fits',              '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f105w_drz_sci.fits'],             'F125W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f125w_drz_SE_sci.fits',              '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f125w_drz_SE_sci.fits',              '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f125w_drz_sci.fits',              '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f125w_drz_sci.fits'],             'F140W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f140w_drz_sci.fits',              '/data1/rowland/all_fits_data/sci_

In [37]:
#amending the wfc list b/c each filter has multiple files and SEx cant handle that
'''

empty_list =[]
for filter,images in wfc_files.items():
    print('hope theres only one filter:',filter)
    for image in images:
        temp_list=[filter, image]
        empty_list.append(temp_list)
print('check this list',empty_list)

#for filter,image in empty_list
    #for image in wfc_files.values():
   #     print('processing this image',image)
  #      ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}
  #  break
    '''

"\n\nempty_list =[]\nfor filter,images in wfc_files.items():\n    print('hope theres only one filter:',filter)\n    for image in images:\n        temp_list=[filter, image]\n        empty_list.append(temp_list)\nprint('check this list',empty_list)\n\n#for filter,image in empty_list\n    #for image in wfc_files.values():\n   #     print('processing this image',image)\n  #      ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}\n  #  break\n    "

In [38]:
#len(empty_list)

In [39]:
'''
no_of_wfcfiles = np.arange(0,len(empty_list),1)


#for index, image in enumerate(empty_list):
#    print('index,item test print:',index, image) #index, [filter,file name]<---each filter has 4 files indiv printouts = good
    #print('test index value:', index) #this is right
for filter,image in empty_list:
    print('processing this image',image)
    print('with this filter:',filter)    
    #NOTE output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files
        #! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}_{index}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}
    break
    '''

"\nno_of_wfcfiles = np.arange(0,len(empty_list),1)\n\n\n#for index, image in enumerate(empty_list):\n#    print('index,item test print:',index, image) #index, [filter,file name]<---each filter has 4 files indiv printouts = good\n    #print('test index value:', index) #this is right\nfor filter,image in empty_list:\n    print('processing this image',image)\n    print('with this filter:',filter)    \n    #NOTE output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files\n        #! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}_{index}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}\n    break\n    "

# Above PSFs were practice apparently, Running SExtractor on El_gordo fits files to get PSFs per filter

In [42]:
elgordo_files = {'F606W': '/data1/rowland/elgordo_F606W.fits', \
              'F625W': '/data1/rowland/elgordo_F625W.fits', \
              'F775W': '/data1/rowland/elgordo_F775W.fits', \
              'F814W': '/data1/rowland/elgordo_F814W.fits', \
              'F850LP': '/data1/rowland/elgordo_F850LP.fits',\
              'F105W': '/data1/rowland/elgordo_F105W.fits',\
              'F125W': '/data1/rowland/elgordo_F125W.fits',\
              'F140W': '/data1/rowland/elgordo_F140W.fits',\
              'F160W': '/data1/rowland/elgordo_F160W.fits',
              'F435W': '/data1/rowland/elgordo_F435W.fits'}


thresh_test = 20.0

for filter,image in elgordo_files.items():
    print('processing this image',image)
    #NOTE output .cat file will save to /home/rowland/leaps folder instead of with other data, fix location or manually move later to sci_n_wht_files
    ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test}
    #break

processing this image /data1/rowland/elgordo_F606W.fits
> 
----- SExtractor 2.19.5 started on 2019-06-17 at 15:37:54 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for elgordo_F606W.fits
----- Measuring from: elgordo_F606W.fits
      "Unnamed" / no ext. header / 8201x8473 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background ma

> Line:  500  Objects:        1 detected /        0 sextracted
> Line:  525  Objects:        2 detected /        0 sextracted
> Line:  550  Objects:        2 detected /        0 sextracted
> Line:  575  Objects:        2 detected /        0 sextracted
> Line:  600  Objects:        2 detected /        0 sextracted
> Line:  625  Objects:        3 detected /        0 sextracted
> Line:  650  Objects:        3 detected /        0 sextracted
> Line:  675  Objects:        3 detected /        0 sextracted
> Line:  700  Objects:        3 detected /        0 sextracted
> Line:  725  Objects:        4 detected /        0 sextracted
> Line:  750  Objects:        8 detected /        0 sextracted
> Line:  775  Objects:        9 detected /        0 sextracted
> Line:  800  Objects:        9 detected /        0 sextracted
> Line:  825  Objects:       13 detected /        0 sextracted
> Line:  850  Objects:       14 detected /        0 sextracted
> Line:  875  Objects:       15 detected /        0 sex

> Line: 3375  Objects:      324 detected /      164 sextracted
> Line: 3400  Objects:      334 detected /      167 sextracted
> Line: 3425  Objects:      342 detected /      171 sextracted
> Line: 3450  Objects:      348 detected /      173 sextracted
> Line: 3475  Objects:      349 detected /      173 sextracted
> Line: 3500  Objects:      354 detected /      176 sextracted
> Line: 3525  Objects:      359 detected /      185 sextracted
> Line: 3550  Objects:      363 detected /      190 sextracted
> Line: 3575  Objects:      368 detected /      193 sextracted
> Line: 3600  Objects:      374 detected /      199 sextracted
> Line: 3625  Objects:      375 detected /      205 sextracted
> Line: 3650  Objects:      381 detected /      209 sextracted
> Line: 3675  Objects:      383 detected /      212 sextracted
> Line: 3700  Objects:      389 detected /      222 sextracted
> Line: 3725  Objects:      395 detected /      223 sextracted
> Line: 3750  Objects:      402 detected /      228 sex

> Line: 6325  Objects:      859 detected /      671 sextracted
> Line: 6350  Objects:      862 detected /      678 sextracted
> Line: 6375  Objects:      868 detected /      682 sextracted
> Line: 6400  Objects:      876 detected /      691 sextracted
> Line: 6425  Objects:      877 detected /      695 sextracted
> Line: 6450  Objects:      881 detected /      702 sextracted
> Line: 6475  Objects:      886 detected /      707 sextracted
> Line: 6500  Objects:      893 detected /      710 sextracted
> Line: 6525  Objects:      896 detected /      713 sextracted
> Line: 6550  Objects:      900 detected /      717 sextracted
> Line: 6575  Objects:      903 detected /      721 sextracted
> Line: 6600  Objects:      904 detected /      722 sextracted
> Line: 6625  Objects:      909 detected /      725 sextracted
> Line: 6650  Objects:      912 detected /      726 sextracted
> Line: 6675  Objects:      915 detected /      729 sextracted
> Line: 6700  Objects:      918 detected /      731 sex

> Setting up background map at line: 1856
> Setting up background map at line: 1920
> Setting up background map at line: 1984
> Setting up background map at line: 2048
> Setting up background map at line: 2112
> Setting up background map at line: 2176
> Setting up background map at line: 2240
> Setting up background map at line: 2304
> Setting up background map at line: 2368
> Setting up background map at line: 2432
> Setting up background map at line: 2496
> Setting up background map at line: 2560
> Setting up background map at line: 2624
> Setting up background map at line: 2688
> Setting up background map at line: 2752
> Setting up background map at line: 2816
> Setting up background map at line: 2880
> Setting up background map at line: 2944
> Setting up background map at line: 3008
> Setting up background map at line: 3072
> Setting up background map at line: 3136
> Setting up background map at line: 3200
> Setting up background map at line: 3264
> Setting up background map at lin

...

> Line: 1350  Objects:       68 detected /        0 sextracted
> Line: 1375  Objects:       72 detected /        0 sextracted
> Line: 1392  Objects:       76 detected /        0 sextracted
> Line: 1400  Objects:       77 detected /        1 sextracted
> Line: 1425  Objects:       79 detected /        1 sextracted
> Line: 1450  Objects:       84 detected /        1 sextracted
> Line: 1475  Objects:       90 detected /        1 sextracted
> Line: 1500  Objects:       92 detected /        3 sextracted
> Line: 1525  Objects:       94 detected /        3 sextracted
> Line: 1550  Objects:       99 detected /        4 sextracted
> Line: 1575  Objects:      103 detected /        4 sextracted
> Line: 1600  Objects:      104 detected /        5 sextracted
> Line: 1625  Objects:      110 detected /        6 sextracted
> Line: 1650  Objects:      117 detected /        6 sextracted
> Line: 1675  Objects:      118 detected /        7 sextracted
> Line: 1700  Objects:      120 detected /        8 sex

> Line: 4300  Objects:      508 detected /      415 sextracted
> Line: 4325  Objects:      508 detected /      417 sextracted
> Line: 4350  Objects:      508 detected /      418 sextracted
> Line: 4375  Objects:      508 detected /      425 sextracted
> Line: 4400  Objects:      508 detected /      428 sextracted
> Line: 4425  Objects:      509 detected /      431 sextracted
> Line: 4450  Objects:      509 detected /      434 sextracted
> Line: 4475  Objects:      509 detected /      438 sextracted
> Line: 4500  Objects:      509 detected /      438 sextracted
> Line: 4525  Objects:      509 detected /      440 sextracted
> Line: 4550  Objects:      509 detected /      441 sextracted
> Line: 4575  Objects:      509 detected /      446 sextracted
> Line: 4600  Objects:      509 detected /      448 sextracted
> Line: 4625  Objects:      509 detected /      449 sextracted
> Line: 4650  Objects:      509 detected /      454 sextracted
> Line: 4675  Objects:      509 detected /      457 sex

> Line: 1241  Objects:       75 detected /        0 sextracted
> Line: 1250  Objects:       78 detected /        1 sextracted
> Line: 1275  Objects:       86 detected /        1 sextracted
> Line: 1300  Objects:       89 detected /        1 sextracted
> Line: 1325  Objects:       92 detected /        1 sextracted
> Line: 1350  Objects:       96 detected /        1 sextracted
> Line: 1375  Objects:       99 detected /        1 sextracted
> Line: 1400  Objects:      106 detected /        2 sextracted
> Line: 1425  Objects:      109 detected /        2 sextracted
> Line: 1450  Objects:      115 detected /        2 sextracted
> Line: 1475  Objects:      120 detected /        4 sextracted
> Line: 1500  Objects:      123 detected /        6 sextracted
> Line: 1525  Objects:      125 detected /        7 sextracted
> Line: 1550  Objects:      134 detected /        8 sextracted
> Line: 1575  Objects:      137 detected /        8 sextracted
> Line: 1600  Objects:      141 detected /        9 sex

> Line: 4375  Objects:      688 detected /      572 sextracted
> Line: 4400  Objects:      688 detected /      576 sextracted
> Line: 4425  Objects:      688 detected /      580 sextracted
> Line: 4450  Objects:      690 detected /      583 sextracted
> Line: 4475  Objects:      690 detected /      587 sextracted
> Line: 4500  Objects:      690 detected /      588 sextracted
> Line: 4525  Objects:      691 detected /      590 sextracted
> Line: 4550  Objects:      692 detected /      593 sextracted
> Line: 4575  Objects:      692 detected /      599 sextracted
> Line: 4600  Objects:      692 detected /      600 sextracted
> Line: 4625  Objects:      692 detected /      604 sextracted
> Line: 4650  Objects:      692 detected /      608 sextracted
> Line: 4675  Objects:      692 detected /      612 sextracted
> Line: 4700  Objects:      692 detected /      614 sextracted
> Line: 4725  Objects:      692 detected /      616 sextracted
> Line: 4750  Objects:      692 detected /      619 sex

> Line: 1550  Objects:      194 detected /       26 sextracted
> Line: 1575  Objects:      200 detected /       30 sextracted
> Line: 1600  Objects:      205 detected /       32 sextracted
> Line: 1625  Objects:      210 detected /       34 sextracted
> Line: 1650  Objects:      217 detected /       34 sextracted
> Line: 1675  Objects:      220 detected /       37 sextracted
> Line: 1700  Objects:      227 detected /       42 sextracted
> Line: 1725  Objects:      235 detected /       47 sextracted
> Line: 1750  Objects:      239 detected /       50 sextracted
> Line: 1775  Objects:      248 detected /       52 sextracted
> Line: 1800  Objects:      255 detected /       57 sextracted
> Line: 1825  Objects:      260 detected /       64 sextracted
> Line: 1850  Objects:      265 detected /       68 sextracted
> Line: 1875  Objects:      272 detected /       69 sextracted
> Line: 1900  Objects:      276 detected /       75 sextracted
> Line: 1925  Objects:      281 detected /       81 sex